<a href="https://colab.research.google.com/github/lmbd92/st1800-st1801-trabajo3-2261/blob/main/Trabajo3Lina_Jose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Integrantes: Lina Beltrán (lbeltra1@eafit.edu.co), Jose Muñoz (jjmunozm@eafit.edu.co)

Materia: Alm. & Recu. de información.

Trabajo 3

Universidad EAFIT - Maestria ciencia en datos y analítica

Trabajo 2 Unidad 2 recuperación de texto

Data: https://github.com/lmbd92/st1800-st1801-trabajo3-2261/blob/main/Trabajo3/twitter.txt

# Abrir archivos desde GitHub

In [13]:
# OJO!!!! Ejecutar este bloque una sola vez (La primera vez), de lo contrario el repo se clonaría varias veces de manera recursiva

!git clone -l -s https://github.com/lmbd92/st1800-st1801-trabajo3-2261.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 45 (delta 19), reused 16 (delta 5), pack-reused 0
Unpacking objects: 100% (45/45), done.
/content/cloned-repo/cloned-repo
LICENSE    Trabajo3		    twitterClimateData.csv.zip
README.md  Trabajo3Lina_Jose.ipynb


**Nota: Para guardar el notebook directamente en Github, dirigirse a File→Save a copy in GitHub**

In [ ]:
from getpass import getpass

# autenticación a github
username = "lmbd92"
!git config --global user.email '$username@gmail.com'
!git config --global user.name '$username'

password = getpass('Password:')

# Agregar, hacer commit y push a cambios
!git add .
!git commit -m 'updating repo from colab'
!git push origin master   

# Preparación de ambiente de trabajo (Dependencias/librerias)

In [ ]:
# Transmisión eficiente de archivos muy grandes desde/hacia almacenamientos como S3, GCS, Azure Blob Storage, HDFS, WebHDFS
!pip install smart-open

# Modelado y procesamiento NLP
!pip install --upgrade gensim
!pip install nltk

# Interpretar el tema en un modelo de temas que se ha ajustado a un corpus
!pip install pyLDAvis

In [6]:
import pandas as pd
import numpy as np
import nltk
# Soporte para expresiones regulares (RE).
import re

In [7]:
# nltk: tokenizador y stopwords

from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stop_words_nltk = set(stopwords.words('english'))
print(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'ag

# Preparación de los datos

In [12]:
# Carga de los datos

path_in = '/content/cloned-repo/Trabajo3/'
path_out = ''
file_name = 'twitterClimateData.csv'

df= pd.read_csv(f"{path_in}{file_name}")
print(df)

                                                    2020                 is  \
0                                                 Winter                has   
1                                                      A               year   
2                                                  HAPPY           HOLIDAYS   
3                                                     10          Questions   
4                                         #climatestrike  #FridaysForFuture   
...                                                  ...                ...   
56287                                                 No               more   
56288                                                 My            #Trumps   
56289                                               Time                 is   
56290                                               This                 is   
56291  This assault on science is an outrage and shou...         --Phillipe   

              the                   year           

# Características y representación

# Análisis frecuencial por token o hashtags

# Nubes de palabras por token o hashtags

# Análisis exploratorio no supervisado utilizando técnicas de LDA

# Clasificación y análisis de sentimientos